In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "2"

#toggle if GPU runs out of memory
#os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2, pickle, time
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scipy.optimize as optimize
import glob
import tifffile as tiff
import pandas as pd
from pathlib import Path

colors = {0: [65,105,225], 
           1: [192,192,192], 2: [255,165,0],
           3: [178,34,34], 4: [112, 41, 99], 
           5: [90,77,65], 6: [0,100,0],
           7: [209,187,130], 8: [205,236,237],
           9: [62,216,221], 10: [237,215,187]}

classes = {11: 0, 12: 0, 
           21: 1, 22: 2, 23: 3, 24: 4, 
           31: 5, 
           41: 5, 42: 5, 43:5,
           52: 5,
           71: 5,
           81: 5, 82: 5,
           90: 5, 95: 5}

www1 = ['block1_conv1/kernel', 'block1_conv1/bias',
        'block1_conv2/kernel', 'block1_conv2/bias',
        'block2_conv1/kernel', 'block2_conv1/bias',
        'block2_conv2/kernel', 'block2_conv2/bias',
        'block3_conv1/kernel', 'block3_conv1/bias',
        'block3_conv2/kernel', 'block3_conv2/bias',
        'block3_conv3/kernel', 'block3_conv3/bias',
        'block4_conv1/kernel', 'block4_conv1/bias',
        'block4_conv2/kernel', 'block4_conv2/bias',
        'block4_conv3/kernel', 'block4_conv3/bias',
        'block5_conv1/kernel', 'block5_conv1/bias',
        'block5_conv2/kernel', 'block5_conv2/bias',
        'block5_conv3/kernel', 'block5_conv3/bias']

tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
#Сверточный слой
def conv_layer(feat, filters, kernel_size, strides, training, reg = None, act_reg = None, 
               name = None, trainable = True, padding = "same"):
    
    conv = tf.compat.v1.layers.Conv2D(filters=filters,
                            kernel_size=kernel_size,
                            strides=strides,
                            padding=padding,
                            name=name,
                            trainable=trainable,
                            kernel_regularizer=reg,
                            activity_regularizer=act_reg)(feat)
    act = tf.nn.leaky_relu(conv, alpha=0.1)
    bn = tf.compat.v1.layers.batch_normalization(act, center=True, scale=True, training=training, renorm=True)
    dropout = tf.compat.v1.layers.dropout(bn, rate = 0.5, training=training)
    return dropout

#Слой транспонированной свертки
def upsamp_layer(feat, filters, kernel_size, strides, training, reg = None, act_reg = None, padding = "same"):
    transpose = tf.keras.layers.Conv2DTranspose(filters=filters,
                                        kernel_size=kernel_size,
                                        strides=strides,
                                        padding=padding,
                                        kernel_regularizer=reg,
                                        activity_regularizer=act_reg)(feat)
    act = tf.nn.leaky_relu(transpose, alpha=0.1)
    bn = tf.compat.v1.layers.batch_normalization(act, center=True, scale=True, training=training, renorm=True)
    dropout = tf.compat.v1.layers.dropout(bn, rate = 0.5, training=training)
    return dropout

def pooling(feat):
    return tf.compat.v1.layers.MaxPooling2D(pool_size=[2, 2], strides=2)(feat)

def unet_pp2(features, labels, mode):
    training = False
    
    features = tf.map_fn(lambda frame: tf.image.per_image_standardization(frame), features)
    reg = None
    act_reg = None
    special_k = 5

    conv000 = tf.compat.v1.layers.Conv2D(64,3,1,padding="same",name='block1_conv1',trainable = False, activation="relu")(features)
    conv001 = tf.compat.v1.layers.Conv2D(64,3,1,padding="same",name='block1_conv2',trainable = False, activation="relu")(conv000)
    pool1 = pooling(conv001)
    conv100 = tf.compat.v1.layers.Conv2D(128,3,1,padding="same",name='block2_conv1',trainable = False, activation="relu")(pool1)
    conv101 = tf.compat.v1.layers.Conv2D(128,3,1,padding="same",name='block2_conv2',trainable = False, activation="relu")(conv100)
    pool2 = pooling(conv101)
    conv200 = tf.compat.v1.layers.Conv2D(256,3,1,padding="same",name='block3_conv1',trainable = False, activation="relu")(pool2)
    conv201 = tf.compat.v1.layers.Conv2D(256,3,1,padding="same",name='block3_conv2',trainable = False, activation="relu")(conv200)
    conv202 = tf.compat.v1.layers.Conv2D(256,3,1,padding="same",name='block3_conv3',trainable = False, activation="relu")(conv201)
    conv203 = tf.compat.v1.layers.Conv2D(256,3,1,padding="same",name='block3_conv4',trainable = False, activation="relu")(conv202)
    pool3 = pooling(conv203)
    conv300 = tf.compat.v1.layers.Conv2D(512,3,1,padding="same",name='block4_conv1',trainable = False, activation="relu")(pool3)
    conv301 = tf.compat.v1.layers.Conv2D(512,3,1,padding="same",name='block4_conv2',trainable = False, activation="relu")(conv300)
    conv302 = tf.compat.v1.layers.Conv2D(512,3,1,padding="same",name='block4_conv3',trainable = False, activation="relu")(conv301)
    conv303 = tf.compat.v1.layers.Conv2D(512,3,1,padding="same",name='block4_conv4',trainable = False, activation="relu")(conv302)
    pool4 = pooling(conv303)
    conv400 = tf.compat.v1.layers.Conv2D(512,3,1,padding="same",name='block5_conv1',trainable = False, activation="relu")(pool4)
    conv401 = tf.compat.v1.layers.Conv2D(512,3,1,padding="same",name='block5_conv2',trainable = False, activation="relu")(conv400)
    conv402 = tf.compat.v1.layers.Conv2D(512,3,1,padding="same",name='block5_conv3',trainable = False, activation="relu")(conv401)
    conv403 = tf.compat.v1.layers.Conv2D(512,3,1,padding="same",name='block5_conv4',trainable = False, activation="relu")(conv402)
    
    upsamp01 = upsamp_layer(conv101, 64, 3, 2, training,reg,act_reg) 
    conv01 = conv_layer(tf.concat((upsamp01, conv001), axis = 3),64,special_k,1,training,reg,act_reg)
    conv01 = conv_layer(conv01,64,3,1,training,reg,act_reg)
    conv01 = conv_layer(conv01,64,3,1,training,reg,act_reg)
 
    upsamp11 = upsamp_layer(conv203, 128, 3, 2, training,reg,act_reg) 
    conv11 = conv_layer(tf.concat((upsamp11, conv101), axis = 3),128,special_k,1,training,reg,act_reg)
    conv11 = conv_layer(conv11,128,3,1,training,reg,act_reg)
    conv11 = conv_layer(conv11,128,3,1,training,reg,act_reg)
 
    upsamp02 = upsamp_layer(conv11, 64, 3, 2, training,reg,act_reg) 
    conv02 = conv_layer(tf.concat((upsamp02, conv001, conv01), axis = 3),64,special_k,1,training,reg,act_reg)
    conv02 = conv_layer(conv02,64,3,1,training,reg,act_reg)
    conv02 = conv_layer(conv02,64,3,1,training,reg,act_reg)
 
    upsamp21 = upsamp_layer(conv303, 256, 3, 2, training,reg,act_reg)
    conv21 = conv_layer(tf.concat((upsamp21, conv203), axis = 3),256,special_k,1,training,reg,act_reg)
    conv21 = conv_layer(conv21,256,3,1,training,reg,act_reg)
    conv21 = conv_layer(conv21,256,3,1,training,reg,act_reg)
 
    upsamp12 = upsamp_layer(conv21, 128, 3, 2, training,reg,act_reg)
    conv12 = conv_layer(tf.concat((upsamp12, conv101, conv11), axis = 3),128,special_k,1,training,reg,act_reg)
    conv12 = conv_layer(conv12,128,3,1,training,reg,act_reg)
    conv12 = conv_layer(conv12,128,3,1,training,reg,act_reg)
 
    upsamp03 = upsamp_layer(conv12, 64, 3, 2, training,reg,act_reg)
    conv03 = conv_layer(tf.concat((upsamp03, conv001, conv01, conv02), axis = 3),64,special_k,1,training,reg,act_reg)
    conv03 = conv_layer(conv03,64,3,1,training,reg,act_reg)
    conv03 = conv_layer(conv03,64,3,1,training,reg,act_reg)
 
    upsamp31 = upsamp_layer(conv403, 512, 3, 2, training,reg,act_reg)
    conv31 = conv_layer(tf.concat((upsamp31, conv303), axis = 3),512,special_k,1,training,reg,act_reg)
    conv31 = conv_layer(conv31,512,3,1,training,reg,act_reg)
    conv31 = conv_layer(conv31,512,3,1,training,reg,act_reg)
 
    upsamp22 = upsamp_layer(conv31, 256, 3, 2, training,reg,act_reg) 
    conv22 = conv_layer(tf.concat((upsamp22, conv203, conv21), axis = 3),256,special_k,1,training,reg,act_reg)
    conv22 = conv_layer(conv22,256,3,1,training,reg,act_reg)
    conv22 = conv_layer(conv22,256,3,1,training,reg,act_reg)
 
    upsamp13 = upsamp_layer(conv22, 128, 3, 2, training,reg,act_reg) 
    conv13 = conv_layer(tf.concat((upsamp13, conv101, conv11, conv12), axis = 3),128,special_k,1,training,reg,act_reg)
    conv13 = conv_layer(conv13,128,3,1,training,reg,act_reg)
    conv13 = conv_layer(conv13,128,3,1,training,reg,act_reg)
 
    upsamp04 = upsamp_layer(conv13, 64, 3, 2, training,reg,act_reg) 
    conv04 = conv_layer(tf.concat((upsamp04, conv001, conv01, conv02, conv03), axis = 3),64,special_k,1,training,reg,act_reg)
    conv04 = conv_layer(conv04,64,3,1,training,reg,act_reg)
    conv04 = conv_layer(conv04,64,3,1,training,reg,act_reg)
 
    decoder1 = tf.compat.v1.layers.Conv2D(filters=6,
                              kernel_size=1,
                              strides=1,
                              padding="same",
                              activation="sigmoid")(conv01)

    decoder2 = tf.compat.v1.layers.Conv2D(filters=6,
                              kernel_size=1,
                              strides=1,
                              padding="same",
                              activation="sigmoid")(conv02)

    decoder3 = tf.compat.v1.layers.Conv2D(filters=6,
                              kernel_size=1,
                              strides=1,
                              padding="same",
                              activation="sigmoid")(conv03)
                              
    decoder4 = tf.compat.v1.layers.Conv2D(filters=6,
                              kernel_size=1,
                              strides=1,
                              padding="same",
                              activation="sigmoid")(conv04)

    votes = tf.stack([decoder1, decoder2, decoder3, decoder4], -1)
    probs = tf.reduce_mean(votes, -1)
    classes = tf.argmax(input=probs, axis=-1)

    predictions = {
      "classes": classes,
      "probabilities": votes,
    }    
        
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

In [4]:
def sharpen(image, P):
    P_lr = cv2.resize(P, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_CUBIC)

    if P_lr.shape[0] < image.shape[0]:
        image = image[:P_lr.shape[0],:, :]
    else:
        P_lr = P_lr[:image.shape[0], :]

    if P_lr.shape[1] < image.shape[1]:
        image = image[:,:P_lr.shape[1], :]
    else:
        P_lr = P_lr[:,:image.shape[1]]

    shape = image.shape
    W = optimize.lsq_linear(image.reshape(shape[0]*shape[1],3), P_lr.flatten(), (0,1), lsq_solver = 'lsmr').x
    V = P_lr-np.sum(W*image, axis = -1)

    V_hr = cv2.resize(V, None, fx=2, fy=2, interpolation = cv2.INTER_CUBIC)
    
    if P.shape[0] < V_hr.shape[0]:
        V_hr = V_hr[:P.shape[0],:, :]
    else:
        P = P[:V_hr.shape[0], :]

    if P.shape[1] < V_hr.shape[1]:
        V_hr = V_hr[:,:P.shape[1], :]
    else:
        P = P[:,:V_hr.shape[1]]
        
    P_hr = P - V_hr
    
    S_hr = cv2.resize(image, None, fx=2, fy=2, interpolation = cv2.INTER_CUBIC)
    I_hr = np.sum(W*S_hr, axis = -1)

    out = np.zeros(S_hr.shape)
    out[:,:,0] = S_hr[:,:,0] + P_hr - I_hr
    out[:,:,1] = S_hr[:,:,1] + P_hr - I_hr
    out[:,:,2] = S_hr[:,:,2] + P_hr - I_hr


    return out

def encode_landCover(img):
    w, h = img.shape
    u,inv = np.unique(img,return_inverse = True)
    colored_img = np.array([classes.get(x, 0) for x in u])[inv].reshape(img.shape)
    return np.uint8(colored_img)

def example_input_fn(generator):
    """ An example input function to pass to predict. It must take a generator as input """

    def _inner_input_fn():
        dataset = tf.compat.v1.data.Dataset.from_generator(generator, output_types=(tf.float32), output_shapes=(128,128,3)).batch(8)
        iterator = dataset.make_one_shot_iterator()
        features = iterator.get_next()
        return features

    return _inner_input_fn

def my_service():
    start, end = 1, 100
    for number in range(start, end):
        yield number


class TFEstimatorServe(object):

    def __init__(self, estimator, input_fn):
        self.data = []
        self.estimator = estimator
        self.input_fn = input_fn(self.data_generator)
        self.results = self.estimator.predict(input_fn=self.input_fn, yield_single_examples=True)
        self.closed = False
    
    def data_generator(self):

        while not self.closed:
            data = self.data.pop(0)
            yield data[:,:,:3]

    def predict(self, data):

        self.data = data
        predictions = []
        for _ in range(len(data)):
            predictions.append(next(self.results)['classes'])
        return predictions

    def close(self):
        self.closed = True
        try:
            next(self.predictions)
        except:
            print("Exception in fast_predict. This is probably OK")

In [5]:
def predict(data, model):
    data = np.asarray(data).astype(np.float32)
    predict_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(x=data, shuffle = False)
    res = list(model.predict(input_fn=predict_input_fn, yield_single_examples=False))
    
    return res[0]
 
def decode_landCover(img):
    w, h = img.shape
    colored_img = np.zeros((w,h,3), dtype = np.uint8)
    for i in range(w):
        for j in range(h):
            colored_img[i,j,:] = colors[img[i,j]]       
    return colored_img
 
def demo(filepath0, classifier, name = None):
    data0 = np.load(filepath0)
 
    img = np.uint8(data0[:,:,:3])
    img = cv2.resize(img, (128,128), interpolation=cv2.INTER_CUBIC)[np.newaxis, ...]
    real = np.uint8(data0[:,:,3])
    real = cv2.resize(real, (128,128), interpolation=cv2.INTER_NEAREST)
    res = predict(img, classifier)

    fig = plt.figure(dpi=128)
    ax = fig.add_subplot(1, 3, 1)
    imgplot = plt.imshow(decode_landCover(np.uint8(res['classes'][0])))
    ax = fig.add_subplot(1, 3, 2)
    imgplot = plt.imshow(cv2.cvtColor(img[0],cv2.COLOR_BGR2RGB))
    ax = fig.add_subplot(1, 3, 3)
    imgplot = plt.imshow(decode_landCover(real))
    if name != None:
        img = decode_landCover(np.uint8(res['classes'][0]))[:,:,::-1]
        cv2.imwrite('/gdrive/My Drive/images/' + name + '.png', img)

FOLDER = './chkp/unet_pp/'

my_checkpointing_config = tf.estimator.RunConfig(
    save_checkpoints_steps = 2048000,
    keep_checkpoint_max = 18,
    log_step_count_steps = 100)

ws = tf.estimator.WarmStartSettings(ckpt_to_initialize_from='./models/vgg19',vars_to_warm_start=www1)
model = tf.estimator.Estimator(unet_pp2, model_dir = FOLDER, config = my_checkpointing_config, warm_start_from=ws)

INFO:tensorflow:Using config: {'_model_dir': './chkp/unet_pp/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 2048000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 18, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [7]:
file_name

'./data/test/Yakutsk\\LE07_L1TP_121017_20010809_20170204_01_T1_'

In [14]:
channel_names = ['B1.tif', 'B2.tif', 'B3.tif', 'B4.tif', 'B5.tif', 'B6_VCID_1.tif', 'B7.tif', 'B8.tif']
folder = './2021/data/test/'
folders = []
files = []
raw_file_names = []

#Select desitred coordinates and city
city = "Yakutsk"
file_name = glob.glob("./data/test/"+city+"/*.tif")[0][:-6]
for i in range(len(channel_names)):
    files.append(file_name + channel_names[i])

    
    
# x1,x2 = 1500, 3500 #SLC
# x1,x2 = 500, 2100 #SanAnt
# x1,x2 = 2500, 4500 #Tulsa
# x1,x2 = 2000,3920 #LA
# x1,x2 = 1700,1400 #NY
# x1,x2 = 5000,3000 #Portland
# x1,x2 = 4900,1400 #Denver
x1,x2 = 1000, 4000 #Yakutsk

y1 = x1+1280
y2 = x2+1280

full_image_with_pop_R = tiff.imread(files[0])[x1:y1,x2:y2]
full_image_with_pop_G = tiff.imread(files[1])[x1:y1,x2:y2]
full_image_with_pop_B = tiff.imread(files[2])[x1:y1,x2:y2]
full_image_with_pop_P = tiff.imread(files[7])[x1*2:y1*2,x2*2:y2*2]
#full_image_with_pop_T = tiff.imread('./data/TilesTest/pop/' + city + '.tif')[x1:y1,x2:y2]

full_image_with_pop = sharpen(np.stack((full_image_with_pop_R, full_image_with_pop_G, full_image_with_pop_B), axis = 2), 
                              full_image_with_pop_P)

cv2.imwrite('./images/unet/out'+city+'_orig.png', full_image_with_pop)

imgs = []
for i in range(20):
    for j in range(20):
        imgs += [full_image_with_pop[128*i:128*(i+1), 128*j:128*(j+1), :]]

del full_image_with_pop

fast_predict = TFEstimatorServe(model, example_input_fn)
out = fast_predict.predict(imgs)

fast_predict.close()
del imgs

out_img = np.zeros((2560,2560,3))
out_encoded = np.zeros((2560,2560))

ct = 0
for i in range(20):
    for j in range(20):
        out_img[128*i:128*(i+1), 128*j:128*(j+1)] = decode_landCover(out[ct])[:,:,::-1]
        out_encoded[128*i:128*(i+1), 128*j:128*(j+1)] = out[ct]
        ct += 1
        
cv2.imwrite('./images/unet/out'+city+'.png', out_img) 
cv2.imwrite('./images/unet/out'+city+'_encoded.png', out_encoded)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./chkp/unet_pp/model.ckpt-71519
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Exception in fast_predict. This is probably OK


True